In [1]:
import pandas as pd

# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [94]:
df = pd.read_csv('ml-latest-small/ratings.csv')

In [80]:
users = (
    df.groupby('userId')
    .count()
    .reset_index()[df_temp.movieId > 100]
    .merge(df.groupby('userId').max()[['timestamp']],how='left',on='userId')
    .merge(df.groupby('userId').min()[['timestamp']],how='left',on='userId')
)
users.rename(columns={'movieId': 'rating_count'}, inplace=True)

users['life_avg_hours'] = users.iloc[:,-2].sub(users.iloc[:,-1])/60/60
users[['userId','rating_count','life_avg_hours']].sort_values(by='life_avg_hours', ascending=False)

,userId,rating_count,life_avg_hours
136,380,1063,143274.905833
208,547,2391,139391.815278
157,427,294,132380.348889
2,15,1700,130942.637778
243,624,1735,127081.251944
...,...,...,...
42,110,120,0.318889
77,217,104,0.272500
118,316,140,0.254444
239,609,140,0.235556


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов. 

Необходимо сформировать две таблицы:
 - таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
 - аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [95]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [96]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [97]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [98]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [90]:
clients_no_address = (
    rzd.merge(auto,how='outer',on='client_id')
    .merge(air,how='outer',on='client_id')
    .fillna(0)
                     )
clients_no_address #таблица без адреса клиента

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [93]:
clients = client_base.merge(clients_no_address,on='client_id',how='outer')
clients #таблица с адресом клиента

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы.  Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
 - У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
 - Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
 - Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

> У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
 
1. удалить дубликаты
2. оставить first/last координаты
3. если набор координат также содержит время получения координаты, можно попробовать ориентироваться на связывание по этому времени с записями о пройденных этапах (например, время создания записи о покупке/прохождении этапа пути и т.п.)

> Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

1. возможно, нужно отсечь выбросы координат (глюки GPS и т.п.), если не предполагается, что пользователь в процессе прохождения пути работы с приложением может значительно перемещаться
2. можно их отсортировать и оставить только N граничных точек, если приложение предполагает движение из точки А в точку Б
 
> Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Количество уникальных/неуникальных координат: как много перемещается пользователь
2. Разница между граничными координатами - позволит определить размах перемещений пользователя
3. Найти пользователей с отличающимися от средних координатами - они либо географичеси удалены от основной массы пользователей, либо используют fake GPS
4. Связать пользователей с данными по географии с учётом ареала координат